Thêm Thư viện

In [32]:
import math
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import VotingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import GridSearchCV
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold


Load data từ file BikeSharingDemand

In [33]:
# Load dữ liệu từ file CSV
dt = pd.read_csv("BikeSharingDemand.csv")
print ( "Hiển thị 5 dòng đầu của tập data\n ", dt.head())
print ( "Xem kiểu dữ liệu của từng thuộc tính ", dt.dtypes)
print ( "Xem thông tin của dữ liệu ", dt.info())
print ( "Kiểm tra giá trị thiếu ",dt.isnull().sum())

Hiển thị 5 dòng đầu của tập data
          datetime  season  holiday  workingday  weather  temp   atemp  \
0  1/1/2011 0:00       1        0           0        1  9.84  14.395   
1  1/1/2011 1:00       1        0           0        1  9.02  13.635   
2  1/1/2011 2:00       1        0           0        1  9.02  13.635   
3  1/1/2011 3:00       1        0           0        1  9.84  14.395   
4  1/1/2011 4:00       1        0           0        1  9.84  14.395   

   humidity  windspeed  casual  registered  count  
0        81        0.0       3          13     16  
1        80        0.0       8          32     40  
2        80        0.0       5          27     32  
3        75        0.0       3          10     13  
4        75        0.0       0           1      1  
Xem kiểu dữ liệu của từng thuộc tính  datetime       object
season          int64
holiday         int64
workingday      int64
weather         int64
temp          float64
atemp         float64
humidity        int64
windsp

Chú thích cho tập dữ liệu  <br>
datetime: cho biết thời gian theo dịnh dạng "%m/%d/%Y %H:%M:%hS  <br> 
season: cho biết mùa  <br> 
1: mùa xuân  <br>
2: mùa hè  <br>
3: mùa thu  <br>
4: mùa đông  <br>
workingday: cho biết có phải là ngày làm việc không bao gồm cả cuối tuần và ngày lễ  <br> 
1: ngày làm việc  
0: ngày không làm việc 
weather: Cho biết thời tiết  <br>
1: Trời trong, có mây, có mây rải rác  <br>
2: Sương mù + Đầy mây, sương mù + ít mây, sương mù  <br>
3: Tuyết nhẹ, mưa nhẹ + sấm sét + mây rải rác, mưa nhẹ + mây rải rác  <br>
4: Mưa lớn + mảnh băng + sấm sét + sương mù, tuyết + sương mù  <br>
temp: Nhiệt độ thực tế tính theo độ Celsius (độ C)  <br>
atemp: Nhiệt độ mà con người cảm thấy tính theo độ Celsius (độ C) <br>
humidity: Độ ẩm tương đối <br>
windspeed: Tốc độ gió <br>
casual: Số lượng xe thuê mà không có đăng ký dịch vụ <br>
registered: Số lượng xe thuê mà có đăng ký dịch vụ <br>
count: Tổng số lượng xe được thuê <br>

Xử lý Data

In [34]:
# tạo cột year, month, day, hour,weekday từ cột datetime
dt["datetime"] = pd.to_datetime(dt["datetime"])
dt["year"] = dt["datetime"].dt.year
dt["month"] = dt["datetime"].dt.month
dt["weekday"] = dt["datetime"].dt.weekday
dt["hour"] = dt["datetime"].dt.hour
print(" Hiển thị 5 dòng đầu của tập data\n",  dt.head())
print(" Xem kiểu dữ liệu của từng thuộc tính",  dt.dtypes)
print(" Xem thông tin của dữ liệu",  dt.info())
print(" Kiểm tra giá trị thiếu", dt.isnull().sum())

 Hiển thị 5 dòng đầu của tập data
              datetime  season  holiday  workingday  weather  temp   atemp  \
0 2011-01-01 00:00:00       1        0           0        1  9.84  14.395   
1 2011-01-01 01:00:00       1        0           0        1  9.02  13.635   
2 2011-01-01 02:00:00       1        0           0        1  9.02  13.635   
3 2011-01-01 03:00:00       1        0           0        1  9.84  14.395   
4 2011-01-01 04:00:00       1        0           0        1  9.84  14.395   

   humidity  windspeed  casual  registered  count  year  month  weekday  hour  
0        81        0.0       3          13     16  2011      1        5     0  
1        80        0.0       8          32     40  2011      1        5     1  
2        80        0.0       5          27     32  2011      1        5     2  
3        75        0.0       3          10     13  2011      1        5     3  
4        75        0.0       0           1      1  2011      1        5     4  
 Xem kiểu dữ liệu của 

Chú thích các thuộc tính mới  <br>
month: tháng (1-12)  <br>
year: năm (2011, 2012)  <br>
hour: giờ trong ngày (0-23 giờ)  <br>
weekday: ngày trong tuần  <br> 
0: thứ 2  <br>
1: thứ 3  <br>
2: thứ 4  <br>
3: thứ 5  <br>
4: thứ 6  <br>
5: thứ 7  <br>
6: chủ nhật  <br>

Chia Dữ tập dữ liệu theo Hold-Out <br>
Test = 1/3 , train = 2/3  <br>

In [35]:
# Chia dữ liệu thành X và y
X = dt.drop(columns=["casual", "registered", "count", "datetime"])
print(X)
y = dt["count"] 
print(y)
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=1 / 3, random_state=42
)
print(X_train)
print(X_test)

       season  holiday  workingday  weather   temp   atemp  humidity  \
0           1        0           0        1   9.84  14.395        81   
1           1        0           0        1   9.02  13.635        80   
2           1        0           0        1   9.02  13.635        80   
3           1        0           0        1   9.84  14.395        75   
4           1        0           0        1   9.84  14.395        75   
...       ...      ...         ...      ...    ...     ...       ...   
10881       4        0           1        1  15.58  19.695        50   
10882       4        0           1        1  14.76  17.425        57   
10883       4        0           1        1  13.94  15.910        61   
10884       4        0           1        1  13.94  17.425        61   
10885       4        0           1        1  13.12  16.665        66   

       windspeed  year  month  weekday  hour  
0         0.0000  2011      1        5     0  
1         0.0000  2011      1        5   

Xây dụng mô hình bằng phương pháp tổng hợp mô hình (Bagging Average) <br>
Với mô hình cơ sở là RandomForestRegressor (rừng ngẫu nhiên)  <br>

In [36]:
tree = RandomForestRegressor()
# Tạo 1 tập hợp gồm 10 mô hình cây hồi quy
baggingTree = BaggingRegressor(estimator=tree,n_estimators=10,random_state=42)
# huấn luyện mô hình 
baggingTree.fit(X_train, y_train)
# Dự đoán kết quả
y_pred = baggingTree.predict(X_test)
# Đánh giá mô hình
err_DT = mean_squared_error(y_test, y_pred)
print("RandomForestRegressor MSE =", str(err_DT))
rmse_err_DT = math.sqrt(err_DT)
print("RandomForestRegressor RMSE = " + str(round(rmse_err_DT, 3)))

RandomForestRegressor MSE = 1771.9371726718855
RandomForestRegressor RMSE =42.094


Xây dụng mô hình bằng phương pháp tổng hợp mô hình (Bagging Average)<br>
Với mô hình cơ sở là DecisionTreeRegressor (cây hồi quy)  <br>

In [37]:
tree = DecisionTreeRegressor()
# Tạo 1 tập hợp gồm 10 mô hình cây hồi quy
baggingTree = BaggingRegressor(estimator=tree, n_estimators=10, random_state=42)
# huấn luyện mô hình 
baggingTree.fit(X_train, y_train)
# Dự đoán kết quả
y_pred = baggingTree.predict(X_test)
# Đánh giá mô hình
err_DT = mean_squared_error(y_test, y_pred)
print("DecisionTreeRegressor MSE =", str(err_DT))
rmse_err_DT = math.sqrt(err_DT)
print("DecisionTreeRegressor RMSE = " + str(round(rmse_err_DT, 3)))

DecisionTreeRegressor MSE = 1836.5793603227091
DecisionTreeRegressor RMSE = 42.855


Huấn luyện mô hình

Xây dụng mô hình bằng phương pháp tổng hợp mô hình (Bagging Average) <br>
Với mô hình cơ sở là LinearRegression (Hồi Quy Tuyến tính)  <br>

In [38]:
tree = LinearRegression()
# Tạo 1 tập hợp gồm 10 mô hình cây hồi quy
baggingTree = BaggingRegressor(estimator=tree, n_estimators=10, random_state=42)
# huấn luyện mô hình 
baggingTree.fit(X_train, y_train)
# Dự đoán kết quả
y_pred = baggingTree.predict(X_test)
# Đánh giá mô hình
err_DT = mean_squared_error(y_test, y_pred)
print("LinearRegression MSE =", str(err_DT))
rmse_err_DT = math.sqrt(err_DT)
print("LinearRegression RMSE = " + str(round(rmse_err_DT, 3)))

LinearRegression MSE = 19850.0021534245
LinearRegression RMSE = 140.89


Xây dụng mô hình bằng phương pháp tổng hợp mô hình (Bagging Average) <br>
Với mô hình cơ sở là KNeighborsRegressor (KNN) 

In [39]:
knn = KNeighborsRegressor(n_neighbors=9, p=1, weights="distance")
# Tạo 1 tập hợp gồm 10 mô hình hồi quy tuyến tính
baggingLM = BaggingRegressor(estimator=knn, n_estimators=10, random_state=42)
# huấn luyện mô hình
baggingLM.fit(X_train, y_train)
# dụ đoán kết quả
y_pred = baggingLM.predict(X_test)
# đánh giá mô hình 
err_LM = mean_squared_error(y_test, y_pred)
print("KNeighborsRegressor MSE =", str(err_LM))
rmse_err_LM = math.sqrt(err_LM)
print("KNeighborsRegressor RMSE  =" + str(round(rmse_err_LM, 3)))

KNeighborsRegressor MSE = 12251.952401635606
KNeighborsRegressor RMSE =110.689


Chia tập dữ liệu theo Kfold

Load Data

In [40]:
# Load dữ liệu từ file CSV
dt = pd.read_csv("./BikeSharingDemand.csv")
print ( "Hiển thị 5 dòng đầu của tập data\n ", dt.head())
print ( "Xem kiểu dữ liệu của từng thuộc tính ", dt.dtypes)
print ( "Xem thông tin của dữ liệu ", dt.info())
print ( "Kiểm tra giá trị thiếu ",dt.isnull().sum())

Hiển thị 5 dòng đầu của tập data
          datetime  season  holiday  workingday  weather  temp   atemp  \
0  1/1/2011 0:00       1        0           0        1  9.84  14.395   
1  1/1/2011 1:00       1        0           0        1  9.02  13.635   
2  1/1/2011 2:00       1        0           0        1  9.02  13.635   
3  1/1/2011 3:00       1        0           0        1  9.84  14.395   
4  1/1/2011 4:00       1        0           0        1  9.84  14.395   

   humidity  windspeed  casual  registered  count  
0        81        0.0       3          13     16  
1        80        0.0       8          32     40  
2        80        0.0       5          27     32  
3        75        0.0       3          10     13  
4        75        0.0       0           1      1  
Xem kiểu dữ liệu của từng thuộc tính  datetime       object
season          int64
holiday         int64
workingday      int64
weather         int64
temp          float64
atemp         float64
humidity        int64
windsp

Xử lý data

In [41]:
# tạo cột year, month, day, hour,weekday từ cột datetime
dt["datetime"] = pd.to_datetime(dt["datetime"])
dt["year"] = dt["datetime"].dt.year
dt["month"] = dt["datetime"].dt.month
dt["weekday"] = dt["datetime"].dt.weekday
dt["hour"] = dt["datetime"].dt.hour
print ( "Hiển thị 5 dòng đầu của tập data\n ", dt.head())
print ( "Xem kiểu dữ liệu của từng thuộc tính ", dt.dtypes)
print ( "Xem thông tin của dữ liệu ", dt.info())
print ( "Kiểm tra giá trị thiếu ",dt.isnull().sum())

Hiển thị 5 dòng đầu của tập data
               datetime  season  holiday  workingday  weather  temp   atemp  \
0 2011-01-01 00:00:00       1        0           0        1  9.84  14.395   
1 2011-01-01 01:00:00       1        0           0        1  9.02  13.635   
2 2011-01-01 02:00:00       1        0           0        1  9.02  13.635   
3 2011-01-01 03:00:00       1        0           0        1  9.84  14.395   
4 2011-01-01 04:00:00       1        0           0        1  9.84  14.395   

   humidity  windspeed  casual  registered  count  year  month  weekday  hour  
0        81        0.0       3          13     16  2011      1        5     0  
1        80        0.0       8          32     40  2011      1        5     1  
2        80        0.0       5          27     32  2011      1        5     2  
3        75        0.0       3          10     13  2011      1        5     3  
4        75        0.0       0           1      1  2011      1        5     4  
Xem kiểu dữ liệu của t

Chia tập dữ liệu theo kfold 

In [42]:
# Chia dữ liệu thành X và y
X = dt.drop(columns=["casual", "registered", "count", "datetime"])
print(X)
y = dt["count"] 
print(y)
# Số lượng fold
n_splits = 10

# Khởi tạo KFold
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

       season  holiday  workingday  weather   temp   atemp  humidity  \
0           1        0           0        1   9.84  14.395        81   
1           1        0           0        1   9.02  13.635        80   
2           1        0           0        1   9.02  13.635        80   
3           1        0           0        1   9.84  14.395        75   
4           1        0           0        1   9.84  14.395        75   
...       ...      ...         ...      ...    ...     ...       ...   
10881       4        0           1        1  15.58  19.695        50   
10882       4        0           1        1  14.76  17.425        57   
10883       4        0           1        1  13.94  15.910        61   
10884       4        0           1        1  13.94  17.425        61   
10885       4        0           1        1  13.12  16.665        66   

       windspeed  year  month  weekday  hour  
0         0.0000  2011      1        5     0  
1         0.0000  2011      1        5   

Chia tập dữ liệu theo kfold và Xây dựng mô hình rừng ngẫu nhiên

In [43]:


# Danh sách để lưu trữ MSE và RMSE cho từng mô hình
mse_results = []
rmse_results = []

# Lặp qua các fold
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    ## Bagging
    # Mô hình cơ sở là Decision Tree
    tree = RandomForestRegressor()
    # Tạo mô hình Bagging với 10 mô hình cơ sở
    baggingTree = BaggingRegressor(estimator=tree, n_estimators=10, random_state=42)
    # Huấn luyện mô hình
    baggingTree.fit(X_train, y_train)
    # Dự đoán kết quả
    y_pred = baggingTree.predict(X_test)
    # Đánh giá mô hình
    mse = mean_squared_error(y_test, y_pred)
    rmse = math.sqrt(mse)
    mse_results.append(mse)
    rmse_results.append(rmse)

# Tính trung bình của MSE và RMSE từ các fold
avg_mse = sum(mse_results) / len(mse_results)
avg_rmse = sum(rmse_results) / len(rmse_results)

# In ra kết quả trung bình của MSE và RMSE
print("RandomForestRegressor Average MSE = ", avg_mse)
print("RandomForestRegressor  Average RMSE = ", avg_rmse)


RandomForestRegressor Average MSE = 1642.104306437011
RandomForestRegressor  Average RMSE = 40.48955890539255


Chia tập dữ liệu theo kfold và Xây dựng mô hình cây hồi quy

In [44]:
# Danh sách để lưu trữ MSE và RMSE cho từng mô hình
mse_results = []
rmse_results = []

# Lặp qua các fold
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    ## Bagging
    # Mô hình cơ sở là Decision Tree
    tree = DecisionTreeRegressor()
    # Tạo mô hình Bagging với 10 mô hình cơ sở
    baggingTree = BaggingRegressor(estimator=tree, n_estimators=10, random_state=42)
    # Huấn luyện mô hình
    baggingTree.fit(X_train, y_train)
    # Dự đoán kết quả
    y_pred = baggingTree.predict(X_test)
    # Đánh giá mô hình
    mse = mean_squared_error(y_test, y_pred)
    rmse = math.sqrt(mse)
    mse_results.append(mse)
    rmse_results.append(rmse)

# Tính trung bình của MSE và RMSE từ các fold
avg_mse = sum(mse_results) / len(mse_results)
avg_rmse = sum(rmse_results) / len(rmse_results)

# In ra kết quả trung bình của MSE và RMSE
print("DecisionTreeRegressor Average MSE = ", avg_mse)
print("DecisionTreeRegressor  Average RMSE = ", avg_rmse)

DecisionTreeRegressor Average MSE =  1837.3079577311626
DecisionTreeRegressor  Average RMSE =  42.800427332501116


Chia tập dữ liệu theo kfold và huấn luyện mô hình với hồi quy tuyến tính

In [45]:

# Danh sách để lưu trữ MSE và RMSE cho từng mô hình
mse_results = []
rmse_results = []

# Lặp qua các fold
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    ## Bagging
    # Mô hình cơ sở là Decision Tree
    tree = LinearRegression()
    # Tạo mô hình Bagging với 10 mô hình cơ sở
    baggingTree = BaggingRegressor(estimator=tree, n_estimators=10, random_state=42)
    # Huấn luyện mô hình
    baggingTree.fit(X_train, y_train)
    # Dự đoán kết quả
    y_pred = baggingTree.predict(X_test)
    # Đánh giá mô hình
    mse = mean_squared_error(y_test, y_pred)
    rmse = math.sqrt(mse)
    mse_results.append(mse)
    rmse_results.append(rmse)

# Tính trung bình của MSE và RMSE từ các fold
avg_mse = sum(mse_results) / len(mse_results)
avg_rmse = sum(rmse_results) / len(rmse_results)

# In ra kết quả trung bình của MSE và RMSE
print("LinearRegression Average MSE = ", avg_mse)
print("LinearRegression Average RMSE = ", avg_rmse)


LinearRegression Average MSE =  20088.30280659077
LinearRegression Average RMSE =  141.70723795235048


Chia tập dữ liệu theo kfold và huấn luyện mô hình với KNN

In [46]:

# Danh sách để lưu trữ MSE và RMSE cho từng mô hình
mse_results = []
rmse_results = []

# Lặp qua các fold
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    ## Bagging
    # Mô hình cơ sở là Decision Tree
    tree = KNeighborsRegressor(n_neighbors=9, p=1, weights="distance")
    # Tạo mô hình Bagging với 10 mô hình cơ sở
    baggingTree = BaggingRegressor(estimator=tree, n_estimators=10, random_state=42)
    # Huấn luyện mô hình
    baggingTree.fit(X_train, y_train)
    # Dự đoán kết quả
    y_pred = baggingTree.predict(X_test)
    # Đánh giá mô hình
    mse = mean_squared_error(y_test, y_pred)
    rmse = math.sqrt(mse)
    mse_results.append(mse)
    rmse_results.append(rmse)

# Tính trung bình của MSE và RMSE từ các fold
avg_mse = sum(mse_results) / len(mse_results)
avg_rmse = sum(rmse_results) / len(rmse_results)

# In ra kết quả trung bình của MSE và RMSE
print("KNeighborsRegressor MSE = ", avg_mse)
print("KNeighborsRegressor Average RMSE = ", avg_rmse)


KNeighborsRegressor MSE =  11456.63966169181
KNeighborsRegressor Average RMSE =  106.97962755001348
